## Import Processed Data

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
from re import sub
from decimal import Decimal
import numpy as np
import re

import tqdm
tqdm.monitor_interval = 0

import os
import importlib.util

current_file_path = os.path.abspath(os.path.join("__file__" ,"../.."))
import_data_path = os.path.join(current_file_path,'src','data')

spec = importlib.util.spec_from_file_location("import_clean_csv", os.path.join(import_data_path,"import_clean_data.py"))
import_data = importlib.util.module_from_spec(spec)
spec.loader.exec_module(import_data)

data = import_data.import_clean_csv()

In [2]:
data.keys()

dict_keys(['tmdb_cast_credit', 'tmdb_crew_credit', 'tmdb_movie_genres', 'tmdb_movie_keywords', 'tmdb_movie_main', 'tmdb_movie_production_companies', 'tmdb_movie_production_countries', 'tmdb_movie_spoken_languages'])

In [3]:
df = data['tmdb_movie_main']

missing_movies = df[(df.budget < 1) | (df.revenue < 1)].reindex(columns=['budget', 'revenue','release_date'])
missing_movies.reset_index(level=[0,1], inplace=True)
missing_movies.sort_index(inplace=True)
len(missing_movies)

1574

In [4]:
def row_scrape_movie_data(row):
    missing_movie_dict = {}
    url = 'https://www.the-numbers.com/'
    
    url2, movie_url = "",""
    row.budget, row.revenue, row.movie_url = np.nan, np.nan, np.nan
    title, release_date = row.title, row.release_date
    title_search_url = title.replace(' ', '+')
     
    #movie title search result page url
    query = '{}search?searchterm={}&searchtype=allmatches'.format(url,title_search_url)
    search_page = requests.get(query)
    search_soup = BeautifulSoup(search_page.content, 'html.parser')

    urls = [] #to keep track of urls in search_page
    tables = search_soup.find_all('div', id='page_filling_chart')

    if 'No movie match found' in tables[1].find('p').get_text():
        return row

    # if there is more than one movie returned in search, pick the one that matches release date year
    for link in tables[1].find_all('a'):
        if "/movie" in link.get('href') or "/daily" in link.get('href'):
            urls.append(link.get('href'))

    if not urls or len(urls)<=1: #if no urls are found
        return row
    else:
        for i in range(0,len(urls)):
            pattern = re.compile(r'.+/daily/(.+)$')
            if 'daily' in urls[i]:
                web_release_date = dt.strptime(re.match(pattern,urls[i]).group(1), '%Y/%m/%d')
            else:
                continue
            if web_release_date.year in range(release_date.year-3,release_date.year+3):
                #if date matches, then the next href link will be the correct link
                #for financial details of the movie
                url2 = urls[i+1]
                break
        else:
            return row

    movie_url = url + url2

    if url2:
        #financial details extracted from movie page
        movie_page = requests.get(movie_url)
        movie_soup = BeautifulSoup(movie_page.content, 'html.parser')

        #scrape worldwide box office revenue
        table = movie_soup.find('table', id='movie_finances')
        tds = table.find_all('td')
        box_office = []
        for i in range(len(tds)):
            if "Box Office" in tds[i].get_text():
                money = tds[i+1].get_text()
                box_office.append(Decimal(sub(r'[^\d.]', '', money)))
        
        if box_office:
            revenue = max(box_office)
        else:
            revenue = np.nan
            
        #scrape production budget
        heading = movie_soup.find('h2', text='Movie Details')
        tds = heading.parent.find_all('td')
        for i, td in enumerate(tds):
            if re.match('^production.+budget.+', td.get_text().lower()):
                budget = Decimal(sub(r'[^\d.]', '', tds[i+1].get_text()))
                break
        else:
            
            budget = np.nan

    row.budget = budget
    row.revenue = revenue
    row.movie_url = movie_url

    return row



In [5]:
wrong_titles = {9396: "Crocodile Dundee 2", 9644: "Loaded Weapon 1", 1011: "Richie Rich", 11658: "Tae Guik Gi: The Brotherhood of War", 
    367961: "Savva. Serdtse voyna", 25353: 'La véritable histoire du Chat Botté', 290864: 'Kung Fu Killer', 
    30379: 'Megiddo: Omega Code 2', 46435: 'Topsy Turvy', 12154: '3 Men and a Baby', 91586: 'Insidious Chapter 2'}

In [6]:
for movie_id, title in wrong_titles.items():
    mask = missing_movies.movie_id == movie_id
    missing_movies.loc[mask,'title'] = title

In [7]:
date_mask = missing_movies.movie_id == 380097
missing_movies.loc[date_mask,'release_date'] = dt(2014,12,31)
missing_movies['movie_url'] = np.nan

In [8]:
#missing_movie_data = scrape_movie_data(missing_movies.iloc[:10])
tqdm.tqdm_notebook().pandas()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [10]:
found_movies = missing_movies.progress_apply(row_scrape_movie_data, axis=1)

HBox(children=(IntProgress(value=0, max=1574), HTML(value='')))

In [13]:
found_movies.to_csv('found_movies.csv', index=False)

In [24]:
found_movies[found_movies.isnull().any(axis=1)]

,movie_id,title,budget,revenue,release_date,movie_url
8,16858,All That Jazz,NaN,37823676,1979-12-20,https://www.the-numbers.com//movie/All-That-Ja...
11,161795,Déjà Vu,NaN,6321997,1998-04-22,https://www.the-numbers.com//movie/Deja-Vu#tab...
14,330770,Evolution,NaN,49587,2015-09-14,https://www.the-numbers.com//movie/Evolution-(...
15,9433,The Edge,NaN,34099255,1997-09-06,https://www.the-numbers.com//movie/Edge-The#ta...
20,25769,Carriers,NaN,6808883,2009-09-04,https://www.the-numbers.com//movie/Carriers#ta...
62,53953,The Tooth Fairy,NaN,NaN,2006-08-08,NaN
64,7504,Earth,NaN,528972,1998-09-16,https://www.the-numbers.com//movie/Earth#tab=s...
65,593,Solaris,NaN,NaN,1972-03-20,NaN
67,27983,"Yours, Mine and Ours",NaN,25912624,1968-04-24,https://www.the-numbers.com//movie/Yours-Mine-...
81,280391,Why I Did (Not) Eat My Father,NaN,NaN,2015-04-08,NaN


In [51]:
found_movies = found_movies[(found_movies.budget > 1)|(found_movies.revenue > 1)]

In [52]:
found_movies[found_movies.isnull().any(axis=1)]

,movie_id,title,budget,revenue,release_date,movie_url


In [27]:
original_movie_df = data['tmdb_movie_main'].reset_index()

In [53]:
merged = original_movie_df.merge(right=found_movies, how='left', on = ['movie_id','title'])

In [54]:
merged['budget'] = np.max(merged[['budget_x', 'budget_y']], axis=1)
merged['revenue'] = np.max(merged[['revenue_x', 'revenue_y']], axis=1)
merged['release_date'] = np.max(merged[['release_date_x', 'release_date_y']], axis=1)
merged = merged.drop(labels=['budget_x', 'budget_y','revenue_x', 'revenue_y', 'release_date_x', 'release_date_y', 'movie_url'], axis=1)


In [30]:
merged.to_csv('tmdb_movies_final.csv', index=False)

In [63]:
#original_movie_df[(original_movie_df.budget > 1) & (original_movie_df.revenue > 1)]
what = merged[(merged.budget > 1)|(merged.revenue > 1)]
what[what.release_date.isnull()]

,movie_id,title,original_language,overview,popularity,runtime,status,tagline,vote_average,vote_count,budget,revenue,release_date
